# NAF / ROME Link Factory
* Utiliser les diverses données pour calculer une table de correspondance rome / naf

## Lancement
Les cellules peuvent être lancées automatiquement ( _run all cells_ ), l'exécution s'arrêtera automatiquement après avoir crée le fichier output: ../outputs/andi_rome2naf_|date|.csv


## Autres idées non exploitées:
- pousser certains domaines naf manuellement reliés aux romes
- exploiter les codes OGR (données insuffisantes)
- obtenir des données plus larges, sur une pluse grande durée (cf DPAE)
- intégrer nombre d'entreprises et leur taille moyenne (sur tout le territoire) (cf. sirene/insee)
- plus dynamique: intégrer variance, écart type, ...
- ...

### Obtention des labels et autres données

In [1]:
## Chargement des labels naf, rome, ogr
from IPython.display import display, HTML
import pandas as pd
import numpy as np

# NAF
naf_labels = pd.read_csv('../ressources/list_NAF_LBB.csv', sep='|', encoding="utf-8")
naf_labels.columns = ['nafdot', 'naf', 'label']
print(f"Obtained {len(naf_labels)} NAF labels")
# display(HTML(naf_labels.head(5).to_html()))

# ROME
rome_labels = pd.read_csv('../ressources/liste_rome_LBB.csv', sep=',', encoding="utf-8")
rome_labels.columns = ['rome', 'rome_1', 'rome_2', 'rome_3', 'label', 'slug']
print(f"Obtained {len(rome_labels)} ROME labels")
# display(HTML(rome_labels.head(5).to_html()))

# Chargement des statistiques d'emploi
emploi_rome_naf = pd.read_csv('../ressources/contrats_30j.csv', sep=',', encoding="utf-8")[['ROME', 'APE700', 'nb_embauches']]
emploi_rome_naf.columns = ['rome', 'naf', 'embauches']
# display(HTML(emploi_rome_naf.head(5).to_html()))
        
    
# Calcul des ratios
naf_embauches = emploi_rome_naf[['naf', 'embauches']].groupby('naf').agg(
    embauches_total_n=pd.NamedAgg(column='embauches', aggfunc=sum)
)
rome_embauches = emploi_rome_naf[['rome', 'embauches']].groupby('rome').agg(
    embauches_total_r=pd.NamedAgg(column='embauches', aggfunc=sum)
)


naf_i = (
emploi_rome_naf
    .merge(naf_embauches, on='naf')
    .merge(rome_embauches, on='rome')
)
naf_i['ratio_naf'] = ( naf_i.embauches / naf_i.embauches_total_n ) * 100
naf_i['ratio_rome'] = ( naf_i.embauches / naf_i.embauches_total_r ) * 100

naf_i = naf_i.sort_values(by=['ratio_naf'], ascending=False)
naf_i.columns


Obtained 732 NAF labels
Obtained 531 ROME labels


Index(['rome', 'naf', 'embauches', 'embauches_total_n', 'embauches_total_r',
       'ratio_naf', 'ratio_rome'],
      dtype='object')

### Fonction de calcul du tableau de correspondance ROME vers NAF

In [2]:
# Rome=>Naf in function form
def rome2nafz(rome, df_in, output_size=15):
    """
    Generate naf list using input rome code and specific dataframe:
    ['rome', 'naf', 'embauches', 'embauches_total_n', 'embauches_total_r', 'ratio_naf', 'ratio_rome']
    
    - First a table is generated listing the naf codes who employ the highest percentage
    (from their total recruitments) of the specified rome code, a score is specified with
    a binning technique
    - Then the NAF codes who also appear in the list of where the specified rome code tends
    to work at are boosted
    - Finaly the result is filtered keeping only those with a score of 1 or higher, and from those
    only the first 15 are kept (can be changed by parameter)
    """
    df_nf = (
    df_in.loc[df_in.rome == rome]
         .sort_values(by=['ratio_naf'], ascending=False)
    )
    df_nf['cumsum'] =  100 - df_nf['ratio_rome'].cumsum()
    df_nf = df_nf.loc[df_nf['cumsum'] > 25]
    df_nf['score'] = pd.to_numeric(pd.cut(
        df_nf['ratio_naf'],
        bins=[1, 2, 5, 10, 50, 100],
        labels=[0, 1, 2, 3, 4]
    ))
    
    df_rs = (
    df_in.loc[naf_i.rome == rome ]
         .sort_values(by=['ratio_rome'], ascending=False)
    )
    df_rs['cumsum'] =  100 - df_rs['ratio_rome'].cumsum()
    df_rs = df_rs.head(10)
    
    df_nf.loc[df_nf['naf'].isin(df_rs['naf']), 'score'] += 1
    df_nf = df_nf.loc[df_nf['score'].ge(1)]
    
    return (
        df_nf.sort_values(by=['score'], ascending=False)
             .head(output_size)
    )[['rome', 'naf', 'score']]

### Vérification de la fonction

In [8]:
# Function Test
ROME = 'D1101'
print(f"Table ROME=>NAF pour \"{rome_labels.loc[(rome_labels['rome'] == ROME, 'label')].iloc[0]}\"")
df  = rome2nafz(ROME, naf_i)
df.merge(naf_labels, on='naf')[['naf', 'label','score']]
assert(len(df) == 15)
df.head(5)

Table ROME=>NAF pour "Boucherie"


,rome,naf,score
12720,D1101,4722Z,4.0
12883,D1101,1013B,3.0
12562,D1101,1011Z,3.0
12696,D1101,4632A,3.0
12716,D1101,4711D,2.0


### Calcul de pour l'ensemble des codes ROME

In [4]:
result_table = {}
for index, row in rome_labels.iterrows():
    result_table[row.rome] = rome2nafz(str(row.rome), naf_i).to_dict('r')
    print(f"row {index} : {len(result_table[row.rome])} Naf codes for Rome {row.rome} \"{rome_labels.loc[(rome_labels['rome'] == row.rome, 'label')].iloc[0]}\" ")

row 0 : 15 Naf codes for Rome A1101 "Conduite d'engins agricoles et forestiers" 
row 1 : 8 Naf codes for Rome A1201 "Bûcheronnage et élagage" 
row 2 : 2 Naf codes for Rome A1202 "Entretien des espaces naturels" 
row 3 : 15 Naf codes for Rome A1203 "Entretien des espaces verts" 
row 4 : 2 Naf codes for Rome A1204 "Protection du patrimoine naturel" 
row 5 : 3 Naf codes for Rome A1205 "Sylviculture" 
row 6 : 8 Naf codes for Rome A1301 "Conseil et assistance technique en agriculture" 
row 7 : 4 Naf codes for Rome A1302 "Contrôle et diagnostic technique en agriculture" 
row 8 : 13 Naf codes for Rome A1303 "Ingénierie en agriculture et environnement naturel" 
row 9 : 15 Naf codes for Rome A1401 "Aide agricole de production fruitière ou viticole" 
row 10 : 13 Naf codes for Rome A1402 "Aide agricole de production légumière ou végétale" 
row 11 : 8 Naf codes for Rome A1403 "Aide d'élevage agricole et aquacole" 
row 12 : 6 Naf codes for Rome A1404 "Aquaculture" 
row 13 : 11 Naf codes for Rome A1

row 112 : 0 Naf codes for Rome D1509 "Management de département en grande distribution" 
row 113 : 2 Naf codes for Rome E1101 "Animation de site multimédia" 
row 114 : 1 Naf codes for Rome E1102 "Ecriture d'ouvrages, de livres" 
row 115 : 15 Naf codes for Rome E1103 "Communication" 
row 116 : 6 Naf codes for Rome E1104 "Conception de contenus multimédias" 
row 117 : 4 Naf codes for Rome E1105 "Coordination d'édition" 
row 118 : 15 Naf codes for Rome E1106 "Journalisme et information média" 
row 119 : 6 Naf codes for Rome E1107 "Organisation d'évènementiel" 
row 120 : 2 Naf codes for Rome E1108 "Traduction, interprétariat" 
row 121 : 4 Naf codes for Rome E1201 "Photographie" 
row 122 : 0 Naf codes for Rome E1202 "Production en laboratoire cinématographique" 
row 123 : 2 Naf codes for Rome E1203 "Production en laboratoire photographique" 
row 124 : 1 Naf codes for Rome E1204 "Projection cinéma" 
row 125 : 15 Naf codes for Rome E1205 "Réalisation de contenus multimédias" 
row 126 : 11 Naf

row 222 : 15 Naf codes for Rome H1404 "Intervention technique en méthodes et industrialisation" 
row 223 : 0 Naf codes for Rome H1501 "Direction de laboratoire d'analyse industrielle" 
row 224 : 15 Naf codes for Rome H1502 "Management et ingénierie qualité industrielle" 
row 225 : 15 Naf codes for Rome H1503 "Intervention technique en laboratoire d'analyse industrielle" 
row 226 : 9 Naf codes for Rome H1504 "Intervention technique en contrôle essai qualité en électricité et électronique" 
row 227 : 1 Naf codes for Rome H1505 "Intervention technique en formulation et analyse sensorielle" 
row 228 : 11 Naf codes for Rome H1506 "Intervention technique qualité en mécanique et travail des métaux" 
row 229 : 3 Naf codes for Rome H2101 "Abattage et découpe des viandes" 
row 230 : 15 Naf codes for Rome H2102 "Conduite d'équipement de production alimentaire" 
row 231 : 2 Naf codes for Rome H2201 "Assemblage d'ouvrages en bois" 
row 232 : 7 Naf codes for Rome H2202 "Conduite d'équipement de fabr

row 320 : 8 Naf codes for Rome I1602 "Maintenance d'aéronefs" 
row 321 : 12 Naf codes for Rome I1603 "Maintenance d'engins de chantier, levage, manutention et de machines agricoles" 
row 322 : 15 Naf codes for Rome I1604 "Mécanique automobile et entretien de véhicules" 
row 323 : 6 Naf codes for Rome I1605 "Mécanique de marine" 
row 324 : 14 Naf codes for Rome I1606 "Réparation de carrosserie" 
row 325 : 8 Naf codes for Rome I1607 "Réparation de cycles, motocycles et motoculteurs de loisirs" 
row 326 : 0 Naf codes for Rome J1101 "Médecine de prévention" 
row 327 : 0 Naf codes for Rome J1102 "Médecine généraliste et spécialisée" 
row 328 : 1 Naf codes for Rome J1103 "Médecine dentaire" 
row 329 : 1 Naf codes for Rome J1104 "Suivi de la grossesse et de l'accouchement" 
row 330 : 0 Naf codes for Rome J1201 "Biologie médicale" 
row 331 : 0 Naf codes for Rome J1202 "Pharmacie" 
row 332 : 15 Naf codes for Rome J1301 "Personnel polyvalent des services hospitaliers" 
row 333 : 3 Naf codes for 

row 434 : 10 Naf codes for Rome L1203 "Art dramatique" 
row 435 : 4 Naf codes for Rome L1204 "Arts du cirque et arts visuels" 
row 436 : 1 Naf codes for Rome L1301 "Mise en scène de spectacles vivants" 
row 437 : 15 Naf codes for Rome L1302 "Production et administration spectacle, cinéma et audiovisuel" 
row 438 : 1 Naf codes for Rome L1303 "Promotion d'artistes et de spectacles" 
row 439 : 9 Naf codes for Rome L1304 "Réalisation cinématographique et audiovisuelle" 
row 440 : 6 Naf codes for Rome L1401 "Sportif professionnel" 
row 441 : 2 Naf codes for Rome L1501 "Coiffure et maquillage spectacle" 
row 442 : 5 Naf codes for Rome L1502 "Costume et habillage spectacle" 
row 443 : 6 Naf codes for Rome L1503 "Décor et accessoires spectacle" 
row 444 : 6 Naf codes for Rome L1504 "Éclairage spectacle" 
row 445 : 8 Naf codes for Rome L1505 "Image cinématographique et télévisuelle" 
row 446 : 5 Naf codes for Rome L1506 "Machinerie spectacle" 
row 447 : 9 Naf codes for Rome L1507 "Montage audio

### Écriture du fichier CSV et fin du traitement

In [5]:
# Write CSV File
import csv
import time
output = f"../outputs/andi_rome2naf_{time.strftime('%Y%m%d')}.csv"
i = 0
with open(output, 'w') as file:
    writer = csv.DictWriter(
        file,
        delimiter=',',
        quotechar='"',
        quoting=csv.QUOTE_MINIMAL,
        fieldnames=['rome', 'rome_label', 'naf', 'naf_label', 'score']
    )
    writer.writeheader()
    for rome, naflist in result_table.items():
        for data in naflist:
            data['rome_label'] = rome_labels.loc[(rome_labels['rome'] == rome, 'label')].iloc[0]
            data['naf_label'] = naf_labels.loc[(naf_labels['naf'] == data['naf'], 'label')].iloc[0]
            writer.writerow(data)
            i += 1
print(f'Wrote {i} rows to {output}')

# Crash to end automatic cell execution
raise SystemExit("Fin traitement automatique")  
    

Wrote 3026 rows to ../outputs/andi_rome2naf_20191203.csv


SystemExit: Fin traitement automatique

/home/pieterjan/.local/share/virtualenvs/andi-data-CDydVwxE/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Code historique - documentation

In [ ]:
# Legacy Code

# df_ok['score'] = 0
# df_ok.loc[df_cumsum < 15, 'score'] = 5
# df_ok['score'] = pd.cut(
 #   df_ok['cumsum'],
 #   bins=[0, 15, 30, 45, 60, 100],
 #   labels=[5, 4, 3, 2, 1]
#)
#df_ok

#df_base['score'] = 0
#df_base.loc[df_base['cumsum'].ge(0), 'score'] = 0
#df_base.loc[df_base['cumsum'].ge(15).lt(30), 'score'] = 1
#df_base.loc[df_base['cumsum'].ge(30).lt(45), 'score'] = 2
#df_base.loc[df_base['cumsum'].ge(45).lt(60), 'score'] = 3
#df_base.loc[df_base['cumsum'].ge(60), 'score'] = 4
#df_base

In [ ]:
def display_result(rome, table, naf_labels):
    print(f"Résultat(s) pour {rome} - \"{rome_labels.loc[(rome_labels['rome'] == rome, 'label')].iloc[0]}\"")
    df = pd.DataFrame(table[rome])
    df = df.merge(naf_labels, on='naf')[['naf', 'label','score']]
    display(HTML(df.head(15).to_html()))


display_result('A1303', result_table, naf_labels)
display_result('M1607', result_table, naf_labels)
# rome2nafz(t1, naf_i)

In [ ]:
# Compute best nafs for rome
ROME = 'M1705'

print(f"Table ROME=>NAF pour \"{rome_labels.loc[(rome_labels['rome'] == ROME, 'label')].iloc[0]}\"")

# Basic order based on employment percentage
# Use rome recruitment % by naf ratio, stop @ 75% of total rome pool
df_base = (
naf_i.loc[naf_i.rome == ROME]
    .sort_values(by=['ratio_naf'], ascending=False)
)
df_base['cumsum'] =  100 - df_base['ratio_rome'].cumsum()
df_base = df_base.loc[df_base['cumsum'] > 25]
df_base['score'] = pd.to_numeric(pd.cut(
    df_base['ratio_naf'],
    bins=[1, 2, 5, 10, 50, 100],
    labels=[0, 1, 2, 3, 4]
))

# Boosting scores with rome orientations
# If naf is in top 10 of rome destinations, increment score
df_rs = (
naf_i.loc[naf_i.rome == ROME ]
    .sort_values(by=['ratio_rome'], ascending=False)
)
df_rs['cumsum'] =  100 - df_rs['ratio_rome'].cumsum()
df_rs = df_rs.head(10)
df_base.loc[df_base['naf'].isin(df_rs['naf']), 'score'] += 1
df_base = df_base.loc[df_base['score'].ge(1)]

# Display
(
df_base
    .merge(naf_labels, on='naf')
    .sort_values(by=['score'], ascending=False)
    [['rome', 'naf', 'label','score']]
).head(15)